# Fonctions pour le calcul avec du Bayésien naïf pour les variables catégorielles

## Lecture et nettoyage des données (code fourni)

In [1]:
using CSV, DataFrames, Gadfly, Distributions, StatsBase

┌ Info: Loading DataFrames support into Gadfly.jl
└ @ Gadfly C:\Users\farme\.julia\packages\Gadfly\09PWZ\src\mapping.jl:228


In [2]:
train = CSV.read("ODtrain.csv")

dropmissing!(train, disallowmissing=true)

n = size(train,1)
p = size(train,2)

str_mode = ["Voiture", "Transport Collectif", "Hybride", "Actif", "Autre"]

d = Array{String}(undef,n)

for i=1:length(str_mode)
   
    ind = train[:D_Mode] .== i
    d[ind] .= str_mode[i] 
    
end

train[:D_Mode_str] = d

categorical!(train, :D_Mode_str);

## Code provenant du fichier Maximum de la vraisemblance
À mettre dans une catégorie à part dans le rapport plz

In [3]:
## COPIED FROM MAX VRAISEMBLANCE
k=5
struct TestSet
    train::DataFrame
    test::DataFrame
end

function getTestSet(initialSet::DataFrame)
    n = length(initialSet[:D_Mode])
    nTest = convert(Int64, round(0.20 * n))
    index = rand(0:n-nTest+1)
    return TestSet(vcat(initialSet[1:index-1, :], initialSet[index+nTest:n, :]), initialSet[index:index+nTest-1, :])
    
    a = [1,2,3]
    b= view(a, 1:2)
    c= vcat(view(a, 1:1), view(a, 3:3))
end
function validate(estimates::Array{Int64}, realValue::Array{Int64})
    n = length(realValue)
    nEach = [0, 0, 0, 0, 0]
    right = [0, 0, 0, 0, 0]
    for i=1:n
        nEach[realValue[i]] += 1
        if estimates[i] == realValue[i]
            right[realValue[i]] += 1
        end
    end
    
    out = ""
    for i=1:k
        #print(str_mode[i], " : ", right[i]/nEach[i]*100, "%\n" ) 
        out *= str_mode[i] * " : " * string(right[i]/nEach[i]*100) * "%\n"
    end
    #print("Total : ", sum(right)/n*100, "%\n" )
    tot = sum(right)/n*100
    out *= "Total : " * string(tot) * "%\n"
    return (total=tot, output=out)
end

validate (generic function with 1 method)

## Fonctions et structures pour le calcul

### Structure qui contient les nombres d'occurences des données nécessaires

Cette structure contient les occurences des différentes variables.<br>
Structure:
- mode[i] contient ni, soit le nombre de déplacements qui ont le mode de transport i
- variables[i, j][l] contient le nombre de déplacements qui ont le mode de transport i et qui ont la catégorie l pour la variable j. Ex: Si la permière variable est P_PERMIS, variables[4, 1][2] est le nombre de déplacement où la personne ne possède pas un permis de conduire et où le mode de transport est actif
- colums[j] contient le symbole associé à la colonne qui contient la variable j
- nCat[j] contient le nombre de catégories pour la variable j

In [4]:
struct Occurences
    mode::Array{Int64}
    variables::Array{Array}
    columns::Array{Symbol}
    nCat::Array{Int64}
end

### Fonctions internes ("privées")

#### getOccurences(trainSet, col, nCat)
Cette fonction construit la structure Occurences pour un modèle donné.
- trainSet: l'ensemble des données d'où on va extraire les occurences. 
- col: le tableau qui contient toutes les variables dont il faut extraire les occurences.
- nCat: le nombre de catégories pour chaque variable explicative.

#### findProb(occ, variable, value)
Cette fonction retourne la probabilité associée à une variable explicative pour une valeur donnée.
- occ: Contient les occurences (n) qui permettent de calculer la probabilité.
- variable: Représente l'indice de la variable pour le champ variables dans occ.
- value représente la valeur de la variable explicative.

#### filterMode(row, occ)
Cette fonction retourne quel mode est le plus probable pour un déplacement. Elle appelle la fonction findProb pour toutes les variables explicatives présentes dans occ.
- row: Le déplacement dont on veut prévoir le mode.
- occ: Contient les occurences (n) qui permettent de calculer la probabilité.


In [5]:
function getOccurences(trainSet::DataFrame, col::Array{Symbol}, nCat::Array{Int64})
    mode = Array{Int64, 1}(undef, k)
    for i=1:k
        mode[i] = size(trainSet[trainSet[:D_Mode] .== i, :], 1)
    end
    
    variables = Array{Array{Int64, 1}, 2}(undef, (k, length(col)))
    
    for i=1:5
        for j=1:length(col)
            variables[i, j] = Array{Int64, 1}(undef, nCat[j])
            for l=1:nCat[j]
                variables[i, j][l] = size(trainSet[(trainSet[:D_Mode] .== i) .& (trainSet[col[j]] .== l), :], 1)
            end
        end
    end
    return Occurences(mode, variables, col, nCat)
end

function findProb(occ::Occurences, variable::Int64, value::Int64)
    q = Array{Float64}(undef, length(occ.mode))
    for i=1:k
        q[i] = log(occ.variables[i, variable][value] + 1) - log(occ.mode[i] + length(occ.nCat[variable]))        
    end
    return q
end

function filterMode(row::DataFrameRow, occ::Occurences)
    p = Array{Float64}(undef, k)
    for i=1:k
       p[i] = log(occ.mode[i] + 1) - log(sum(occ.mode) + length(occ.mode)) 
    end 
    for i=1:length(occ.columns)
        x̃ = row[occ.columns[i]]
        p = p + findProb(occ, i, x̃)
    end
    
     _, ind = findmax(p)
    
    return ind
end

filterMode (generic function with 1 method)

### Fonction "publique"

getPercentRightBayes(col, nCat)
Calcule la validité d'un modèle
- col: le nom de la variable catégorielle .
- le nombre de catégories pour chaque variable.

In [6]:
function getPercentRightBayes(col::Array{Symbol}, nCat::Array{Int64})
    set = getTestSet(train)
    occ = getOccurences(set.train, col, nCat)
    sizeTest = size(set.test, 1)
    est = Array{Int64}(undef, sizeTest)
    for i=1:sizeTest
        est[i] = filterMode(set.test[i, :], occ)
    end
    output = validate(est, set.test[:D_Mode])
    print(output.output)
end

getPercentRightBayes (generic function with 1 method)

## Exemple

In [7]:
getPercentRightBayes(    
    [:P_PERMIS, :P_GRAGE, :D_MOTIF, :P_STATUT],
    [5, 16, 13, 8]
)

Voiture : 79.21740320153235%
Transport Collectif : 47.27988546886185%
Hybride : 0.0%
Actif : 0.12143290831815423%
Autre : 83.71877890841813%
Total : 60.4903809883063%


### Ajout d'une nouvelle variable, la municipalité

In [45]:
d = Array{Int64}(undef,n)

for i=1:length(train[:M_DOMSM])
    mun = train[:M_DOMSM][i];
    if mun == 101 || mun == 102
        d[i] = 1;
    elseif (mun >= 103 && mun <= 112) || (mun >= 120 && mun <= 124)
        d[i] = 2;
    elseif mun >= 113 && mun <= 118
        d[i] = 3;
    elseif mun >= 126 && mun <= 141
        d[i] = 4;
    elseif mun >= 301 && mun <= 310
        d[i] = 5;
    elseif mun >= 401 && mun <= 408
        d[i] = 6;
    elseif mun >= 501 && mun <= 576
        d[i] = 7;
    elseif mun >= 601 && mun <= 691
        d[i] = 8;
    else
        d[i] = 9;
    end
end

train[:M_DOMSM_G] = d

getPercentRightBayes(    
    [:P_PERMIS,  :D_MOTIF, :P_STATUT, :M_DOMSM_G],
    [5, 13, 8, 9]
)

Voiture : 83.10615448161917%
Transport Collectif : 41.59074982480729%
Hybride : 0.0%
Actif : 22.316555488174654%
Autre : 82.75538894095595%
Total : 63.93059222934742%
